[DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Disease Tracking Dashboard

This is a template for your DIY Disease Tracking Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from anytree import Node, RenderTree
from anytree.importer import JsonImporter, DictImporter
import datetime
from IPython.display import clear_output
import ipywidgets as wdg
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import pickle
import requests
import time

import warnings
warnings.filterwarnings('ignore', category=FutureWarning) # , message="Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead.")





In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100
global regions 
regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
global filters
filters={"stratum" : None, # Smallest subgroup a metric can be broken down into e.g. ethnicity, testing pillar
    "age": None, # Smallest subgroup a metric can be broken down into e.g. 15_44 for the age group of 15-44 years
    "sex": None, #  Patient gender e.g. 'm' for Male, 'f' for Female or 'all' for all genders
    "year": None,#2022, #  Epi year of the metrics value (important for annual metrics) e.g. 2020
    "month": None, # Epi month of the metric value (important for monthly metrics) e.g. 12
    "epiweek" :None, # Epi week of the metric value (important for weekly metrics) e.g. 30
    "date" : None, # The date which this metric value was recorded in the format YYYY-MM-DD e.g. 2020-07-20
    "in_reporting_delay_period": None # Boolean indicating whether the data point is considered to be subject to retrospective updates
}

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("mean_occBed_region.json", "rt") as INFILE:
    mean_occBed_region=json.load(INFILE)

with open("leaf_first.json", "rt") as INFILE:
    leaf_data =json.load(INFILE)

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")



    data={}
    for dataset in rawdata:
        for entry in dataset:
            date=entry['date']
            value=entry['metric_value']
            geography=entry['geography']
            if date not in data:
                data[date]={}
            data[date][geography]=value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    print (startdate, ' to ', enddate)

    index=pd.date_range(startdate, enddate, freq='D')
    mean_occBed_region_df=pd.DataFrame(index=index, columns=regions)


    for date, values in data.items():
        for region, value in values.items():
            mean_occBed_region_df.at[parse_date(date), region] = value
    # fill in any remaining "holes" due to missing dates
    mean_occBed_region_df.fillna(0.0, inplace=True)

    return mean_occBed_region_df

    

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df1=wrangle_data(mean_occBed_region) # df is the dataframe for plotting

2020-08-07 00:00:00  to  2024-10-31 00:00:00


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

### using v2

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the UKHSA API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    
    class APIwrapper:
        # class variables shared among all instances
        _access_point="https://api.ukhsa-dashboard.data.gov.uk/v2"       
        _last_access=0.0 # time of last api access
        root = Node("root")
        
        def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
            """ Init the APIwrapper object, constructing the endpoint from the structure
            parameters """
            # build the path with all the required structure parameters. You do not need to edit this line,
            # parameters will be replaced by the actual values when you instantiate an object of the class!
            url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                    f"{geography_type}/geographies/{geography}/metrics/{metric}")
            # our starting API endpoint
            self._start_url=APIwrapper._access_point+url_path
            self._filters=None
            self._page_size=-1
            # will contain the number of items
            self.count=None



        def get_page(self, filters={}, page_size=5):
            """ Access the API and download the next page of data. Sets the count
            attribute to the total number of items available for this query. Changing
            filters or page_size will cause get_page to restart from page 1. Rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365); use the default value 
            for debugging your structure and filters. """
            # Check page size is within range
            if page_size>365:
                raise ValueError("Max supported page size is 365")
            # restart from first page if page or filters have changed
            if filters!=self._filters or page_size!=self._page_size:
                self._filters=filters
                self._page_size=page_size
                self._next_url=self._start_url
            # signal the end of data condition
            if self._next_url==None: 
                return [] # we already fetched the last page
            # simple rate limiting to avoid bans
            curr_time=time.time() # Unix time: number of seconds since the Epoch
            deltat=curr_time-APIwrapper._last_access
            if deltat<0.33: # max 3 requests/second
                time.sleep(0.33-deltat)
            APIwrapper._last_access=curr_time
            # build parameter dictionary by removing all the None
            # values from filters and adding page_size
            parameters={x: y for x, y in filters.items() if y!=None}
            parameters['page_size']=page_size
            # the page parameter is already included in _next_url.
            # This is the API access. Response is a dictionary with various keys.
            # the .json() method decodes the response into Python object (dictionaries,
            # lists; 'null' values are translated as None).
            response = requests.get(self._next_url, params=parameters).json()
            # update url so we'll fetch the next page
            self._next_url=response['next']
            self.count=response['count']
            # data are in the nested 'results' list
            return response['results'] 

        def get_all_pages(self, filters={}, page_size=365):
            """ Access the API and download all available data pages of data. Sets the count
            attribute to the total number of items available for this query. API access rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365), and controls the trade-off
            between time to load a page and number of pages; the default should work well 
            in most cases. The number of items returned should in any case be equal to 
            the count attribute. """
            data=[] # build up all data here
            while True:
                # use get_page to do the job, including the pacing
                next_page=self.get_page(filters, page_size)
                if next_page==[]:
                    break # we are done
                data.extend(next_page)
            return data


        # /themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/{geography_type}/geographies/{geography}/metrics/{metric}
        def add_branch_to_node(self, upper_node=None, link=None):
        
            # to kick off the tere from scratch
            # had some problems with using defualt values in the function so dealing with it here
            if upper_node is None:
                upper_node = APIwrapper.root
            if link is None:
                link = APIwrapper._access_point + '/themes/'
                
            importer = DictImporter()
            curr_time = time.time()  # Unix time: number of seconds since the Epoch
            deltat = curr_time - APIwrapper._last_access
            if deltat < 0.33:  # max 3 requests/second
                time.sleep(0.33 - deltat)
            APIwrapper._last_access = curr_time

            branches = requests.get(link).json()
            if branches:                                                # make this better api stuff, 200?
                for branch in branches:
                    new_node = importer.import_(branch)
                    new_node.parent = upper_node
                    new_node.name = branch['name']                      # Use the value of the "name" dictionary key

                    curr_time = time.time()                             # Unix time: number of seconds since the Epoch
                    deltat = curr_time - APIwrapper._last_access
                    if deltat < 0.33:                                   # max 3 requests/second
                        time.sleep(0.33 - deltat)
                    APIwrapper._last_access = curr_time
                    next_level_link = branch['link']                    # get the link to the next section in the api

                    if "/metrics/" in next_level_link:                  # hacky way to get to find if we are at the bottom of the tree (metric data is next)
                        continue                                        # continue to next branch

                    options_link = requests.get(next_level_link).json() # get the link that returns the options for the next branch
                    options_link = list(options_link[0].values())[0]
                    


                    self.add_branch_to_node(new_node, options_link)     # recursively add the next branch to the tree

            return
    
    # This Builds The Tree !!It Takes Over 50min To Run on Local Machine!!
    # api.add_branch_to_node()
    
    # Load the tree from a pickle file instead
    global root
    # root = api.root
    with open('full_tree_named.pkl', 'rb') as f:
        root = pickle.load(f)

    structure={"theme": "infectious_disease", 
        "sub_theme": "respiratory",
        "topic": "COVID-19", "metric": "COVID-19_healthcare_occupiedBedsRollingMean", "geography_type": "NHS Region"} #, "geography": "England"}
    
    
    mean_occBed_region = []
    for idx, r in enumerate(regions):
        structure["geography"] = r
        api=APIwrapper(**structure)
        mean_occBed_region.append(api.get_all_pages(filters))                    
        print(f"Data points expected: {api.count}")
        print(f"Data points retrieved: {len(mean_occBed_region[idx])}")
        
    with open("mean_occBed_region.json", "wt") as OUTF:
        json.dump(mean_occBed_region, OUTF)
        

    return mean_occBed_region

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df1, df2
    global regions 
    regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
    df1=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # The function needs to be adapted to your graph; you can omit this call
    # in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. If you are 
    # implementing error handling, you can use icons "unlink" or "times" and 
    # change the button text to "Unavailable" when the api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Engage', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Make it so",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

# display(apibutton)

# run all cells before clicking on this button

Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
2020-08-07 00:00:00  to  2024-10-31 00:00:00


## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [ ]:
def update_plot(selected_regions, date_range):
    fig, ax = plt.subplots(figsize=(12, 6))
    selected_regions = list(selected_regions)                   # make a list of the regions to filter by
    start_date, end_date = date_range                           # pull the start and end date out of the tuple              
    df1a = df1.loc[start_date:end_date, selected_regions].copy()               
    df1a.plot(ax=ax)
    ax.set_title("Mean Beds Occupied by Region")
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x', which='minor', rotation=65)
    ax.tick_params(axis='x', which='major', rotation=65)
    ax.set_xlabel('Date')
    ax.set_ylabel('Metric Value')

    # since df is a reasonable size decided to filter data rather than the plot
    # ax.set_xlim([start_date, end_date])   
                        
    # since filtering date by the data, can scale the y-axis based on the date range
    ax.set_ylim([df1a.min().min(), df1a.max().max()+50])
    
    plt.show()
  

reg = wdg.SelectMultiple(
    options=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    value=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    description='Regions',
    disabled=False
)
reg.layout.height = '120px'

dates = pd.date_range(start='2020-08-01', end='2024-10-31', freq='MS')
options = [(i.strftime('%b %Y'), i) for i in dates]
dates_slider = wdg.SelectionRangeSlider(
    options=options,
    index=(0, 50),
    description='Period',
    disabled=False,
    layout=wdg.Layout(width='40%')
)

# forcably prevent the start date equalling the end date. Not very elegant.
def validate_date_range(change):
    start, end = change['new']
    if start.year == end.year and start.month == end.month:
        if start == dates[0]:
            end = dates[1]
        else:
            start = dates[dates.get_loc(end) - 1]
        yr.value = (start, end)

dates_slider.observe(validate_date_range, names='value')



def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=reg.value
    #print(current)
    if current==reg.options[0]:
        other=reg.options[1]
        #print("was reg.options[0]",other)
    else:
        other=reg.options[0]
        #print("was reg.options[1]",other)
    reg.value=(other,) # forces the redraw
    reg.value=current # now we can change it back
    
controls0=wdg.HBox([apibutton, reg, dates_slider]) 

# connect the plotting function and the widget    
graph0=wdg.interactive_output(update_plot, {'selected_regions': reg, 'date_range': dates_slider})

# actually displays the graph
display(controls0, graph0)

Output()

## All Option Titles

I'd heard of tree datastuctures a few years ago and decided to map the whole api to a tree structure. this took nearly an hour on my machine so it is written to a pickle file and loaded from there. The mapping code is included in the function `add_branch_to_node`. Each node has a dictinoary containing the api link and the name, and the function explores all branches down to the metrics data. The code checks when a dropdown is changed and updates the following dropdowns so you can't have parent showing that doesn't belong to a child that is shown in another dropdown.  
I wanted to try pulling the data from the link in the Metrics dictionary, rather than building a link with the structure, so retrieved the pages outside the APIwrapper class.


In [8]:
# load the pickle of the tree (if not loaded from file takes nearly an hour to generate)
with open('full_tree_named.pkl', 'rb') as f:
    root = pickle.load(f)




# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def getMetric_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    leaf_data = requests.get(metric_link, params={ 'page_size': 1800, 'page': 1}).json()
    
    # One-off to save json data to file
    # with open("leaf_first.json", "wt") as OUTF:
    #     json.dump(leaf_data, OUTF)
    
    # wrangle the data and overwrite the dataframe for plotting
    global df2
    df2=wrangle_leaf_data(leaf_data)
    
    refresh_graph()

    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. If you are 
    # implementing error handling, you can use icons "unlink" or "times" and 
    # change the button text to "Unavailable" when the api call fails.
    getMetric_button.icon="check"
    # apibutton.disabled=True

    
getMetric_button=wdg.Button(
    description='Engage', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Make it so",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
getMetric_button.on_click(getMetric_button_callback) # the name of your function inside these brackets




def wrangle_leaf_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    data={}
    for entry in rawdata['results']:
        date=entry['date']
        value=entry['metric_value']
        if date not in data:
            data[date] = value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    leaf_metric_df = pd.DataFrame(index=index, columns=['Metric'])

    for date, value in data.items(): # each entry is a dictionary with cases, admissions and deaths
        pd_date=parse_date(date) # convert to Pandas format
        # do not assume all values are there for every date - if a value is not available, insert a 0.0
        # this is the way you access a specific location in the dataframe - use .loc
        # and put index,column in a single set of [ ]
        leaf_metric_df.loc[date, 'Metric'] = value
    leaf_metric_df.fillna(0.0, inplace=True)

    return leaf_metric_df



def plot_metric(change, title):                                        # revieves a change parameter from the widget to refresh, but not used
    fig, ax = plt.subplots(figsize=(14, 6))             
    df2.plot(ax=ax)
    ax.set_title(title)
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x', which='minor', rotation=80)
    ax.tick_params(axis='x', which='major', rotation=80)
    ax.set_xlabel('Date')
    ax.set_ylabel('Metric Value')

    plt.show()

    
# helper dropdown to change without impact, and update graph. Not displayed in control1
helper=wdg.Dropdown(
    options=['One', 'Two'],
    value='One',
    description='helper',
    disabled=False,
)


def refresh_graph():
    """Change the value of the widget in order to force a redraw of the graph
    since we pull from the API every time with this graph.
    Don't want the useful dropdowns to change as they can trigger a reset and also
    not all dropdowns have >1 choice."""
    current=helper.value
    if current==helper.options[0]:
        other=helper.options[1]
    else:
        other=helper.options[0]
    helper.value=other # forces the redraw
    helper.value=current # now we can change it back



    

# function to create a dropdown widgets, rather than one by one
def create_dropdown(description, options):
    return wdg.Dropdown(
        options=options,
        value=options[0][1] if options else None,
        description=description,
        disabled=False,
        style={'description_width': '90px'},
        layout=wdg.Layout(width='30%')
    )


# problem with changing earlier dropdowns messing up later dropdown values, e.g. changing sub theme after topics were chosen
# so need to update them all
def update_options(dropdown, options):
    dropdown.options = options
    if options:
        dropdown.value = options[0][1]
    else:
        dropdown.value = None

# update each dropdown
def update_subthemes(*args):
    if theme_dd.value is not None:  # it has a value that might not work with a previous selection (not a problem for theme at the moment but future proofing)
        update_options(subTheme_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children)])
        update_topics()

def update_topics(*args):
    if subTheme_dd.value is not None:
        update_options(topic_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children)])
        update_geotypes()

def update_geotypes(*args):
    if topic_dd.value is not None:
        update_options(geoType_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children)])
        update_geographies()

def update_geographies(*args):
    if geoType_dd.value is not None:
        update_options(geographies_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children)])
        update_metrics()

def update_metrics(*args):
    if geographies_dd.value is not None:
        update_options(metrics_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children[geographies_dd.value].children)])

# create the dropdown widgets
theme_dd = create_dropdown('1. Theme:', [(child.name, idx) for idx, child in enumerate(root.children)])
subTheme_dd = create_dropdown('2. Sub Theme:', [])
topic_dd = create_dropdown('3. Topic:', [])
geoType_dd = create_dropdown('4. Geo Type:', [])
geographies_dd = create_dropdown('5. Geographies:', [])
metrics_dd = create_dropdown('6. Metrics:', [])



def update_metric_link(*args):
    global metric_link
    metric_link = root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children[geographies_dd.value].children[metrics_dd.value].link


# observe the dropdowns so downstream resets when a value changes
theme_dd.observe(update_subthemes, names='value')
subTheme_dd.observe(update_topics, names='value')
topic_dd.observe(update_geotypes, names='value')
geoType_dd.observe(update_geographies, names='value')
geographies_dd.observe(update_metrics, names='value')

# Observe all dropdowns to update the metric link
theme_dd.observe(update_metric_link, names='value')
subTheme_dd.observe(update_metric_link, names='value')
topic_dd.observe(update_metric_link, names='value')
geoType_dd.observe(update_metric_link, names='value')
geographies_dd.observe(update_metric_link, names='value')
metrics_dd.observe(update_metric_link, names='value')

update_subthemes(None)

controls1 = wdg.VBox([
    wdg.HBox([theme_dd, subTheme_dd, topic_dd], layout=wdg.Layout(justify_content='space-between', width='100%')),
    wdg.HBox([geoType_dd, geographies_dd, metrics_dd], layout=wdg.Layout(justify_content='space-between', width='100%')), 
    getMetric_button
])

# plot first graph from file data
df2=wrangle_leaf_data(leaf_data) # df is the dataframe for plotting

# connect the plotting function and the widget    
graph1=wdg.interactive_output(plot_metric, {'change' : helper, 'title': wdg.fixed(metrics_dd.label)})   # get the label to send to the plotting function as a title

# actually displays the graph
display(controls1, graph1)




Output()

In [9]:
# def wrangle_leaf_data(rawdata):
#     """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
#     Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
#     def parse_date(datestring):
#         """ Convert a date string into a pandas datetime object """
#         return pd.to_datetime(datestring, format="%Y-%m-%d")



#     data={}
#     for entry in rawdata['results']:
#         date=entry['date']
#         value=entry['metric_value']
#         if date not in data:
#             data[date] = value

#     dates=list(data.keys())
#     dates.sort()

#     startdate=parse_date(dates[0])
#     enddate=parse_date(dates[-1])

#     index=pd.date_range(startdate, enddate, freq='D')
#     leaf_metric_df = pd.DataFrame(index=index, columns=['Metric'])




#     for date, value in data.items(): # each entry is a dictionary with cases, admissions and deaths
#         pd_date=parse_date(date) # convert to Pandas format
#         # do not assume all values are there for every date - if a value is not available, insert a 0.0
#         # this is the way you access a specific location in the dataframe - use .loc
#         # and put index,column in a single set of [ ]
#         leaf_metric_df.loc[date, 'Metric'] = value
#     leaf_metric_df.fillna(0.0, inplace=True)

#     return leaf_metric_df
    
    
    
    

# df2=wrangle_leaf_data(leaf_data)


## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."